In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from utils import load_embeddings
import wandb

/home/ubuntu/anaconda3/envs/unitary_detoxify/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(entity="anitavero", project="finetune")
%env WANDB_SILENT=true

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anitavero. Use `wandb login --relogin` to force relogin


env: WANDB_SILENT=true


In [3]:
ids, embeddings, metadata = load_embeddings('/home/ubuntu/dev/detoxify/jigsaw-toxic-comment-classification-challenge/embeddings/embeddings_t5-large_test_This_text_is_about_{}.pkl')

In [4]:
labels = pd.read_csv('/home/ubuntu/dev/detoxify/jigsaw-toxic-comment-classification-challenge/data/test_labels.csv', dtype={"id": "string"})
classes = list(labels.columns)
classes.remove("id")
y = labels[classes].to_numpy()

In [5]:
mask = (y != -1).any(axis=1)
y_m = y[mask, :]
embeddings_m = embeddings[mask, :]
y_m.shape, embeddings_m.shape

X_train, X_test, y_train, y_test = train_test_split(embeddings_m, y_m, random_state=1)

In [7]:
clf = MLPClassifier(random_state=1, max_iter=100, verbose=True)
mo_clf = MultiOutputClassifier(estimator=clf)#.fit(X_train, y_train)

In [9]:
def train():
    config = wandb.config
    loss = mo_clf.fit(X_train, y_train)  # your model training code here
    wandb.log({"loss": loss})

def predict_prob(embs):
    preds = mo_clf.predict_proba(embs)
    return np.column_stack([p[:, 1] for p in preds])

In [20]:
sweep_config = {
  "name" : "my-sweep",
  "method" : "bayes",
  "metric": {
    "name": "val_loss",
    "goal": "minimize"
  },
  "parameters" : {
    # "epochs" : {
    #   "values" : [10, 20, 50]
    # },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.1
    }
  }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: gwfebgo4
Sweep URL: https://wandb.ai/anitavero/uncategorized/sweeps/gwfebgo4


In [ ]:
count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

In [9]:
from model_eval.eval_predictions import evaluate

In [10]:
# y_probs = predict_prob(X_test)
# y_pred = mo_clf.predict(X_test)
# score = mo_clf.score(X_test, y_test)

evaluate(y_probs, y_test)

{'auc_scores': [0.9332486899501415,
  0.9698520767649705,
  0.9364603929743527,
  0.9578011147486553,
  0.9391207122163853,
  0.9428209870325803],
 'mean_auc': 0.9465506622811809}